# Fact table techniques: Measure categories

We will be analyzing multiple fact table techniques that can be applied to all fact tables.

## Fact table structure

Here, we recap the basics of a fact table:

- Should contain numeric measures produced by operational events in the real world.
- A row should correspond to an event and vice-versa.
- Should always contain foreign keys for each associated dimension, as well as optional degenerate dimension keys for dates and timestamps.
- Should be the primary target of computations and dynamic aggregations for queries.

## Categorizing numerical measures

As we have stated previously, fact tables should contain numerical measures. These measures can be categorized into three types:
1. **Aditive**: Measures that can be summed across any of the dimensions associated with the fact table.
2. **Semi-aditive**: Can be summed across some dimensions but not others. For example, account balances can be summed other dimensions, but you can't sum along time since it's a measure of a point in time.
4. **Non-aditive**: Measures that can't be summed across any dimension. For example, ratios and percentages - If you sum them they don't make sense.

Since non-aditive facts don't add much value to the fact table, and may be used to generate misleading metrics, the question of whether or not to store them physically arises. They're useful for showing the actual value, and for filtering, but thats about it.

### Derived facts
There is a special type of fact called a **derived fact**. These are facts that are derived from other facts. For example, a profit measure can be derived from the difference between revenue and cost. Derived facts can be physically stored in the fact table or calculated on the fly. Calculating it on the fly makes it prone to user errors, as well as it impacts performance. We need to analyze the physical cost of storing the derived fact vs the cost of calculating it on the fly and getting it wrong. Another thing to keep in mind is how often the rules for calculating the derived fact change. If it changes often, it might be better to calculate it on the fly. Views can be used as a middle ground, where the derived fact is calculated on the fly but the view is stored in the database. The view isn't maintained by the users, however the analyst must ensure that all users use the view and not the underlying table.



## Case study: Retail Sales fact table

For our previous case study using the retail sales fact table, we can identify some of the categories we have just discussed:

![Retail Sales Partial Dimensional Model](https://github.com/gustavom2998/engineering_notes/blob/main/books/data_warehouse_toolkit/images/2_1.png?raw=true)

### DuckDB Generated Columns
In DuckDB, when were adding a [generated column](https://duckdb.org/docs/sql/statements/create_table#generated-columns) (which we're using to implement derived facts) - we can choose wether to store it virtually or phyisically unsing the `STORED` keyword. Virtually stored values are calculated every time the data is read. Physically stored values are calculated when the data is written, and stored in the table. By default, generated columns are virtual values, and the type can also be infered by DuckDB. Also, generated columns can't be used with the `INSERT` statement.

In [1]:
import duckdb

db = duckdb.connect("retail_sales.db")

### Profit as a derived fact
The `extended_gross_profit_dollars` can be calculated by subtracting the `extended_revenue_dollars` and `extended_cost_dollars` measures. This is an example of a derived fact. We can calculate this value in multiple steps of our pipeline. It could be generated by the source system, and we could assume that it's correct. We could also calculate it in the ETL process, or in a view built on top of the table loaded with the ETL process. In this case, we will calculate it in LOAD step of our ETL process by making it a column that is calculated when data is inserted into the table.

In [2]:
# Remove old column definition (needs to be populated by ETL)
gross_profit_fact_ddl_delete = """
ALTER TABLE retail_sales 
DROP COLUMN IF EXISTS extended_gross_profit_dollars;
"""

db.execute(gross_profit_fact_ddl_delete)

# Add new column definition - Derived fact implemented using DuckDB Generated Columns
## Obs: ALTER TABLE ADD GENERATED COLUMN IS NOT SUPPORTED IN DUCKDB
gross_profit_fact_ddl_create = """
ALTER TABLE retail_sales 
ADD COLUMN extended_gross_profit_dollars FLOAT
GENERATED ALWAYS AS (extended_sales_dollars - extended_cost_dollars) STORED;
"""


### Gross Margin (%) as a derived fact
We can also introduce a new column called `gross_margin_percent` which can be calculated by diving the `extended_gross_profit_dollars` by `extended_sales_dollars`. This is an example of a non-additive fact. Since we can't sum it, it's a non-additive fact. 


In [3]:
# Add new column - Derived fact implemented using DuckDB Generated Columns
## Obs: ALTER TABLE ADD GENERATED COLUMN IS NOT SUPPORTED IN DUCKDB
gross_margin_fact_ddl_create = """
ALTER TABLE retail_sales 
ADD COLUMN extended_gross_profit_dollars FLOAT
GENERATED ALWAYS AS (extended_gross_profit_dollars / extended_sales_dollars) STORED;
"""

Finally, we add the columns by recreating the table since DuckDB Alter table statements don't support adding generated columns (but this should be added in the near future). If the table was already populated with data, we could copy it into a backup auxiliary table, and then add the calculated column by generating it with a `Create Table As Select` (CTAS) statement.

```sql

In [4]:
# Recreate table with new column definition since alter table for generated columns is not supported
derived_fact_ddl = """
CREATE OR REPLACE TABLE retail_sales (
    date_id INTEGER REFERENCES date_dim(date_id),
    store_id INTEGER REFERENCES store_dim(store_id),
    cashier_id INTEGER REFERENCES cashier_dim(cashier_id),
    product_id INTEGER REFERENCES product_dim(product_id),
    promotion_id INTEGER REFERENCES promotion_dim(promotion_id),
    payment_method_id INTEGER REFERENCES payment_method_dim(payment_method_id),
    pos_transaction_id INTEGER,
    sales_quantity UINTEGER,
    regular_unit_price FLOAT,
    cost_unit_price FLOAT,
    discount_unit_price FLOAT,
    net_unit_price FLOAT GENERATED ALWAYS AS (regular_unit_price - discount_unit_price) VIRTUAL,
    extended_discount_dollars FLOAT GENERATED ALWAYS AS (discount_unit_price * sales_quantity) VIRTUAL,
    extended_sales_dollars FLOAT GENERATED ALWAYS AS (net_unit_price * sales_quantity) VIRTUAL,
    extended_cost_dollars FLOAT GENERATED ALWAYS AS (cost_unit_price * sales_quantity) VIRTUAL,
    extended_gross_profit_dollars FLOAT GENERATED ALWAYS AS (extended_sales_dollars - extended_cost_dollars) VIRTUAL,
    extended_gross_margin_percent FLOAT GENERATED ALWAYS AS (extended_gross_profit_dollars / extended_sales_dollars) VIRTUAL,
    PRIMARY KEY (date_id, store_id, cashier_id, product_id, promotion_id, payment_method_id)
)
"""

db.execute(derived_fact_ddl)

### Non-additive facts
To showcase this behaviour, We can start of by populating our tables with data.

In [5]:
# Import the ETL class and run it
from scripts.etl_retail_sales import RetailSalesETL

load_retail_sales_obj = RetailSalesETL(db)
load_retail_sales_obj.load_retail_sales_data()

In [6]:
# Check the loaded data
db.execute("""SELECT * FROM retail_sales limit 10""").df()


,date_id,store_id,cashier_id,product_id,promotion_id,payment_method_id,pos_transaction_id,sales_quantity,regular_unit_price,cost_unit_price,discount_unit_price,net_unit_price,extended_discount_dollars,extended_sales_dollars,extended_cost_dollars,extended_gross_profit_dollars,extended_gross_margin_percent
0,1,2,1,4,1,2,1,2,1.20,0.88,0.00,1.20,0.00,2.400000,1.760000,0.640000,0.266667
1,0,1,1,1,2,1,2,3,1.24,0.87,0.05,1.19,0.15,3.570000,2.610000,0.960000,0.268908
2,2,2,1,5,2,1,3,6,4.29,2.84,0.08,4.21,0.48,25.260000,17.039999,8.220001,0.325416
3,2,2,2,2,2,2,4,7,4.66,2.91,0.38,4.28,2.66,29.959999,20.370001,9.589998,0.320093
4,0,1,2,3,2,1,5,10,6.88,4.47,1.04,5.84,10.40,58.400002,44.699997,13.700005,0.234589
5,0,1,2,4,2,1,6,4,1.84,1.34,0.24,1.60,0.96,6.400000,5.360000,1.040000,0.162500
6,0,2,1,5,2,1,7,7,4.40,3.10,0.32,4.08,2.24,28.559999,21.699999,6.860001,0.240196
7,0,1,2,5,3,2,8,1,2.08,1.21,0.25,1.83,0.25,1.830000,1.210000,0.620000,0.338798
8,0,1,2,5,3,3,9,4,5.01,2.97,0.21,4.80,0.84,19.200001,11.880000,7.320001,0.381250
9,0,2,2,1,1,2,10,2,0.21,0.15,0.00,0.21,0.00,0.420000,0.300000,0.120000,0.285714


The previously added column `extended_gross_margin_percent` was an example of a non-additive fact. This is because it's a ratio, and it doesn't make sense to sum it. We can see this by trying to sum it the gross margin percent grouping by each store. Percentage values should never be over 100% (or 1 since we're using decimals), but we can see that we got a value over 100%.

In [7]:
# Treat extended_gross_margin_percent as an additive fact
db.execute("""
    SELECT sd.store_name, SUM(rs.extended_gross_margin_percent) as sum_extended_gross_margin_percent
    FROM retail_sales as rs
    JOIN store_dim as sd ON sd.store_id = rs.store_id
    GROUP BY sd.store_name
    ORDER BY 2 DESC
""").df()


,store_name,sum_extended_gross_margin_percent
0,Store 2,7.286086
1,Store 1,5.111445
2,Store 0,3.792631


If however we analyze the average gross margin percent for each store, we can see that it does make sense to calculate the average. We can aggregate on this fact, however we can't simply sum it. The following aggregation tells us that on average Store 1 has a higher profit margin than store 2 and 0.

In [10]:
# Treat extended_gross_margin_percent as an additive fact
db.execute("""
    SELECT sd.store_name, AVG(rs.extended_gross_margin_percent) as avg_extended_gross_margin_percent
    FROM retail_sales as rs
    JOIN store_dim as sd ON sd.store_id = rs.store_id
    GROUP BY sd.store_name
    ORDER BY 2 DESC
""").df()


,store_name,avg_extended_gross_margin_percent
0,Store 1,0.340763
1,Store 2,0.331186
2,Store 0,0.316053


Another example of a non-additive fact is the `regular_unit_price` and `net_unit_price` measures. Summing up unit prices across different dimensions generally doesn't make sense. It generates a number, however it's not a useful number. This number can be calculated quite easily, but in a business sense, it has little meaning. It gives us the for the sum of all the products sold by store, considering only 1 product per transaction (So even if some bought 2 of an item, it's only counted twice).

In [11]:
# Treat extended_gross_margin_percent as an additive fact
db.execute("""
    SELECT sd.store_name, SUM(rs.regular_unit_price) as sum_regular_unit_price
    FROM retail_sales as rs
    JOIN store_dim as sd ON sd.store_id = rs.store_id
    GROUP BY sd.store_name
    ORDER BY 2 DESC
""").df()

,store_name,sum_regular_unit_price
0,Store 2,93.749999
1,Store 0,63.190000
2,Store 1,61.930001


By using the correct metrics, such as `extended_sales_dollars`, we can get a useful number such as the average price per unit sold, which is a useful metric. This metric could be use to build a report that shows. This give us a absolute value for the ammount of money that came into a store. We could improve this even more by using the `extended_gross_profit_dollars` measure, which gives us the absolute value of the profit generated by a store. This is a useful metric since it gives us the actual value of the profit generated by a store, and we could use to see which stores are making the most money - even if they're selling less items.

In [14]:
# Treat extended_gross_margin_percent as an additive fact
db.execute("""
    SELECT 
        sd.store_name, 
        SUM(rs.extended_sales_dollars) as sum_extended_sales_dollars,
        SUM(rs.extended_gross_profit_dollars) as sum_extended_gross_profit_dollars
    FROM retail_sales as rs
    JOIN store_dim as sd ON sd.store_id = rs.store_id
    GROUP BY sd.store_name
    ORDER BY 3 DESC
""").df()

,store_name,sum_extended_sales_dollars,sum_extended_gross_profit_dollars
0,Store 2,506.779999,175.059995
1,Store 1,369.570009,115.270013
2,Store 0,310.950000,110.840000


In [15]:
db.close()